## SITCOM Laser Tracker Analysis: Elevation LUT

In this notebook we look at the measured laser tracker offsets against elevation, and the change in LUTs for different iterations.

Relevant imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.time import Time
import yaml
from numpy.polynomial import Polynomial

from lsst_efd_client import EfdClient
from lsst.sitcom.vandv.lasertracker import utils

%matplotlib inline
%reload_ext lab_black
%reload_ext autoreload
%autoreload 2

## Retrieve relevant data

In [ ]:
client = EfdClient("usdf_efd")
fields = ["dX", "dY", "dZ", "dRX", "dRY", "target"]

start = Time("2024-03-10 19:48:00Z", scale="utc")
end = Time("2024-03-10 22:20:00Z", scale="utc")

offsets_mar09 = await client.select_time_series(
    "lsst.sal.LaserTracker.logevent_offsetsPublish",
    fields,
    start,
    end,
)


start = Time("2024-03-28 04:49:44Z", scale="utc")
end = Time("2024-03-28 06:50:00Z", scale="utc")

offsets_mar28 = await client.select_time_series(
    "lsst.sal.LaserTracker.logevent_offsetsPublish",
    fields,
    start,
    end,
)

# Extract offsets, target elevation, azimuth and rotator angle by component
data_cam_mar28 = utils.extract_component_data(offsets_mar28, "Camera")
data_m2_mar28 = utils.extract_component_data(offsets_mar28, "M2")

# Extract offsets, target elevation, azimuth and rotator angle by component
data_cam_mar09 = utils.extract_component_data(offsets_mar09, "Camera")
data_m2_mar09 = utils.extract_component_data(offsets_mar09, "M2")

## Get current LUT data

This notebook requires that you have the current ts_aos_utils installed in your notebooks folder.
Make sure to git clone it if you don't have it. 

In [ ]:
home_path = os.getenv("HOME")
lut_path = os.path.join(
    home_path,
    "notebooks/ts_aos_utils/python/lsst/ts/aos/utils/data/hexapod/hexapod_lookuptables.yaml",
)

with open(lut_path, "r") as yaml_file:
    lut_data = yaml.safe_load(yaml_file)


lut_path_update = os.path.join(
    home_path,
    "notebooks/ts_config_mttcs/MTHexapod/v3/laser_tracker_m1m3_mass_sim_lut.yaml",
)

with open(lut_path_update, "r") as yaml_file:
    lut_data_update = yaml.safe_load(yaml_file)

## Plot offsets and fitted LUT

### Camera offsets

In [ ]:
plt.figure(figsize=(15, 8))
for idx, field in enumerate(fields[:-1]):
    plt.subplot(2, 3, idx + 1)
    plt.plot(
        data_cam_mar09["target_elevation"],
        data_cam_mar09[field] - np.mean(data_cam_mar09[field]),
        ".-",
        label="Measured - No LUT",
    )
    plt.plot(
        data_cam_mar28["target_elevation"],
        data_cam_mar28[field] - np.mean(data_cam_mar28[field]),
        ".-",
        label="Measured - Updated LUT",
    )

    # Fit values
    new_poly, [residual, _, _, _] = Polynomial.fit(
        90 - data_cam_mar28["target_elevation"],
        data_cam_mar28[field] - np.mean(data_cam_mar28[field]),
        5,
        full=True,
    )
    coefs_fit = np.flip(new_poly.convert().coef)
    plt.plot(
        data_cam_mar28["target_elevation"],
        new_poly(90 - data_cam_mar28["target_elevation"]),
        "--.k",
        label="Update LUT Fit",
    )

    # Format plot
    plt.xlabel("Elevation (deg)")
    if idx >= 3:
        plt.ylabel("deg")
    else:
        plt.ylabel("um")
    plt.title(f"Camera {field} Offsets")
    plt.legend()

plt.tight_layout()

### M2

In [ ]:
plt.figure(figsize=(15, 8))
for idx, field in enumerate(fields[:-1]):
    plt.subplot(2, 3, idx + 1)
    plt.plot(
        data_m2_mar09["target_elevation"],
        data_m2_mar09[field] - np.mean(data_m2_mar09[field]),
        ".-",
        label="Measured - No LUT",
    )
    plt.plot(
        data_m2_mar28["target_elevation"],
        data_m2_mar28[field] - np.mean(data_m2_mar28[field]),
        ".-",
        label="Measured - Updated LUT",
    )

    # Fit values
    new_poly, [residual, _, _, _] = Polynomial.fit(
        90 - data_m2_mar28["target_elevation"],
        data_m2_mar28[field] - np.mean(data_m2_mar28[field]),
        5,
        full=True,
    )
    coefs_fit = np.flip(new_poly.convert().coef)
    plt.plot(
        data_m2_mar28["target_elevation"],
        new_poly(90 - data_m2_mar28["target_elevation"]),
        "--.k",
        label="Update LUT Fit",
    )

    # Format plot
    plt.xlabel("Elevation (deg)")
    if idx >= 3:
        plt.ylabel("deg")
    else:
        plt.ylabel("um")
    plt.title(f"M2 {field} Offsets")
    plt.legend()

plt.tight_layout()

## LUT Comparison

### Camera

In [ ]:
plt.figure(figsize=(15, 8))
for idx, field in enumerate(fields[:-1]):
    plt.subplot(2, 3, idx + 1)

    # Plot current LUT curve
    coefs_current_lut = lut_data["camera_config"]["elevation_coeffs"][idx]
    y_current_lut = np.polyval(
        coefs_current_lut[::-1], data_cam_mar28["target_elevation"]
    )  # Evaluate the polynomial
    plt.plot(
        data_cam_mar28["target_elevation"], -y_current_lut, label=f"-1* Original LUT"
    )

    coefs_current_lut = lut_data_update["camera_config"]["elevation_coeffs"][idx]
    y_updated_lut = np.polyval(
        coefs_current_lut[::-1], data_cam_mar28["target_elevation"]
    )  # Evaluate the polynomial
    plt.plot(
        data_cam_mar28["target_elevation"], -y_updated_lut, label=f"-1* Updated LUT"
    )

    # Format plot
    plt.xlabel("Elevation (deg)")
    if idx >= 3:
        plt.ylabel("deg")
    else:
        plt.ylabel("um")
    plt.title(f"Camera {field} LUT Correction")
    plt.legend()

plt.tight_layout()
plt.savefig("LUT_corrections_comparison_camera.png", dpi=300)

### M2

In [ ]:
plt.figure(figsize=(15, 8))
for idx, field in enumerate(fields[:-1]):
    plt.subplot(2, 3, idx + 1)

    # Plot current LUT curve
    coefs_current_lut = lut_data["m2_config"]["elevation_coeffs"][idx]
    y_current_lut = np.polyval(
        coefs_current_lut[::-1], data_m2_mar28["target_elevation"]
    )  # Evaluate the polynomial
    plt.plot(
        data_m2_mar28["target_elevation"], -y_current_lut, label=f"-1* Original LUT"
    )

    coefs_current_lut = lut_data_update["m2_config"]["elevation_coeffs"][idx]
    y_updated_lut = np.polyval(
        coefs_current_lut[::-1], data_m2_mar28["target_elevation"]
    )  # Evaluate the polynomial
    plt.plot(
        data_m2_mar28["target_elevation"], -y_updated_lut, label=f"-1* Updated LUT"
    )

    # Format plot
    plt.xlabel("Elevation (deg)")
    if idx >= 3:
        plt.ylabel("deg")
    else:
        plt.ylabel("um")
    plt.title(f"M2 {field} LUT corrections")
    plt.legend()

plt.tight_layout()
plt.savefig("LUT_corrections_comparison_M2.png", dpi=300)